In [1]:
# Code to mount the drive 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#For Himanshu's access
import os
path = "/content/drive/MyDrive/CollabProjects/685/catr"
os.chdir(path)

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 3.4 MB 4.1 MB/s 
     |████████████████████████████████| 3.3 MB 53.9 MB/s 
     |████████████████████████████████| 61 kB 603 kB/s 
     |████████████████████████████████| 596 kB 73.0 MB/s 
     |████████████████████████████████| 895 kB 63.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.3 MB/s 
     |████████████████████████████████| 1.2 MB 23.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=2f33a43f3cfd7898da094edae514fcaa78ccdcaa912bcb0a306334a045474f4d
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [5]:
from transformers import ViTModel, ViTConfig, ViTFeatureExtractor,BertTokenizer,BertForMaskedLM
#from transformers import DeiTFeatureExtractor, DeiTModel #AutoFeatureExtractor, DeiTForImageClassificationWithTeacher, 
from sentence_transformers import SentenceTransformer
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import auc, precision_score, recall_score,roc_auc_score
import xgboost as xgb
from PIL import Image
import argparse
import glob
import json
import pandas as pd
import numpy as np
from models import caption
from datasets import coco, utils
from tqdm import tqdm
from configuration import Config
from xgboost import XGBClassifier
import os


ModuleNotFoundError: ignored

In [6]:
##Initializing Variables

prediction_type = "emogen"
checkpoint_name = "BaselineEmogenTestSetResults"

if prediction_type == "sarcasm":
  ##For image
  images_mappings = r'../sarcasm/annotations/captions_test.json'
  f = open(images_mappings)
  image_details = json.load(f)["annotations"]
  image_path = '../sarcasm/test/'

  ##For text
  text_reference_path = r'./IntermediateFineTuning/'
  text_csv_file = checkpoint_name + ".csv"

  ##Classification
  classification_model_file = "../sarcasm/evaluation/Vision_trans/xgb_vis_cls_sarcasm_mmodal.pkl"

else:
  ##For image
  images_mappings = r'./IntermediateFineTuning/'+checkpoint_name+'.json'
  f = open(images_mappings)
  image_details = json.load(f)["annotations"]
  image_path = '../emogen/test/'

  ##For text
  text_reference_path = r'./IntermediateFineTuning/'
  text_csv_file = checkpoint_name + ".csv"

  ##Classification
  classification_model_file = "../emogen/evaluations/Vision_trans/xgb_cls_positive_vit_mmodal.pkl"

In [ ]:
##Extracting test image features


def extract_vision_transformer_feats(image_path, images, nsel_st=None,nsel_end=None):
  """
  Function to extract the features from vision transformers
  """
  #code to get all images present in a certain csv file
  # csv_data=pd.read_csv("/content/drive/MyDrive/685/emogen/Classifier/train/caption/positive_captions.csv")
  img_list = [os.path.join(image_path, image["image"]+'.jpg') for image in images]

  if not nsel_st:
    img_files_list = img_list#glob.glob(image_path+"*")
  else:
    img_files_list = img_list[nsel_st:nsel_end]#glob.glob(image_path+"*")[nsel_st:nsel_end]
  
  # Create image batch array for Vision Transformer
  img_batch = []
  for file in tqdm(img_files_list):
    img = np.asarray(Image.open(file))
    newsize = (240, 240, 3)
    img = np.resize(img,newsize)
    img_batch.append(img)
  print("Creation of image batches to be used for Vision Transformer complete")

  
  # Extract features from ViTModel
  feature_extractor   = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
  model               = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
  #feature_extractor   =  DeiTFeatureExtractor.from_pretrained('facebook/deit-tiny-distilled-patch16-224')
  #model               = DeiTModel.from_pretrained('facebook/deit-tiny-distilled-patch16-224')
  inputs              = feature_extractor(images=img_batch, return_tensors="pt")
  outputs             = model(**inputs)

  # Get image representations and their corresponding hashes i.e. get [CLS] token representation for each image
  img_representations = outputs.last_hidden_state[:,0,:]
  img_hash_li         = []
  for file in img_files_list:
    img_hash_li.append(file.split("/")[-1][:-4])
  
  # Create column names for image dimensions 
  col_img  = ["imdim_"+str(i) for i in list(range(768))]

  # Create a dataframe of image features
  img_data = pd.DataFrame(img_representations.detach().numpy(),columns = col_img)
  img_data['img_hash'] = img_hash_li

  return img_data

img_data = extract_vision_transformer_feats(image_path, image_details)
img_data


100%|██████████| 476/476 [04:52<00:00,  1.63it/s]


Creation of image batches to be used for Vision Transformer complete


Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

In [18]:
def extract_sentence_transformer_feats(reference_file_pth = '/content/drive/MyDrive/685/emogen/Classifier/train/caption/',csv_file = 'positive_captions.csv',lab_assign=1):
  """
  Extract features for emotion related texts
  """
  # Get reference text data
  combined_path = reference_file_pth + csv_file
  #file = open(combined_path)
  #data_json = json.load(file)
  data = pd.read_csv(combined_path)#pd.DataFrame.from_dict(data_json['annotations']).reset_index(drop=True)
  data = data.reset_index(drop=True)
  try:
    del data['Unnamed: 0']
  except:
    pass
  
  # Use Sentence Transformer to extract features
  model = SentenceTransformer('all-mpnet-base-v2')
  sentence_embeddings = model.encode(data['caption'])
  col_text = ["tdim_"+str(i) for i in list(range(768))]

  # Create text feature dataframe
  text_data = pd.DataFrame(sentence_embeddings,columns=col_text)
  text_data['img_hash'] = data['image']
  text_data['label'] = 1

  return text_data

text_data = extract_sentence_transformer_feats(text_reference_path, text_csv_file)


In [19]:
img_data.to_csv("feature_embeddings/image_"+checkpoint_name+".csv")
text_data.to_csv("feature_embeddings/text_"+checkpoint_name+".csv")


In [20]:
print(img_data.shape, text_data.shape)
print(img_data.columns)
img_data = img_data.reset_index()
print(img_data.columns)

(100, 769) (100, 770)
Index(['imdim_0', 'imdim_1', 'imdim_2', 'imdim_3', 'imdim_4', 'imdim_5',
       'imdim_6', 'imdim_7', 'imdim_8', 'imdim_9',
       ...
       'imdim_759', 'imdim_760', 'imdim_761', 'imdim_762', 'imdim_763',
       'imdim_764', 'imdim_765', 'imdim_766', 'imdim_767', 'img_hash'],
      dtype='object', length=769)
Index(['index', 'imdim_0', 'imdim_1', 'imdim_2', 'imdim_3', 'imdim_4',
       'imdim_5', 'imdim_6', 'imdim_7', 'imdim_8',
       ...
       'imdim_759', 'imdim_760', 'imdim_761', 'imdim_762', 'imdim_763',
       'imdim_764', 'imdim_765', 'imdim_766', 'imdim_767', 'img_hash'],
      dtype='object', length=770)


In [21]:
##Loading the model
import pickle
model = pickle.load(open(classification_model_file, "rb"))

In [22]:
##Classification
data = img_data.merge(text_data,how='left',on='img_hash')
# print(data)
# print(data['img_hash'])
#from sklearn.model_selection import train_test_split
# data
X_tst = data.drop(columns=['label', 'img_hash'],axis=1)
y_tst = data['label']
# X_trn, X_tst, y_trn, y_tst = train_test_split(data.drop(columns=['label','img_hash'],axis=1), data['label'], test_size=100)

# X_train = X_trn.values
# y_train = y_trn.values
X_test  = X_tst.values
y_test  = y_tst.values 



print(X_test)
print(y_test)

pred_test = model.predict(X_test)
pred_prob = model.predict_proba(X_test)
accuracy = (sum(pred_test == y_test))*100/len(y_test)
precision = precision_score(y_test,pred_test)
recall    = recall_score(y_test,pred_test)
# auc       = roc_auc_score(y_test,pred_test)

print("ACCURACY => %s" % (accuracy))
print("PRECISION => %s" % (precision))
print("RECALL => %s" % (recall))
print("AUC => %s" % (auc))

[[ 0.00000000e+00 -7.94769377e-02 -1.27268404e-01 ...  8.08127895e-02
  -1.24076903e-02  2.16962933e-03]
 [ 1.00000000e+00 -1.33500621e-01 -1.97358765e-02 ... -1.42440507e-02
   6.63501397e-03  6.86811609e-03]
 [ 2.00000000e+00 -1.96786657e-01  5.76654188e-02 ... -2.92988829e-02
   6.25205133e-03 -1.89012568e-02]
 ...
 [ 9.70000000e+01 -2.60333776e-01  2.73992836e-01 ...  6.74678758e-02
   4.95787151e-03 -5.12312865e-03]
 [ 9.80000000e+01 -1.73445120e-01 -2.66374685e-02 ... -2.21731979e-03
   2.29747742e-02 -1.42124956e-02]
 [ 9.90000000e+01 -5.16118333e-02  1.44802630e-01 ... -1.61033180e-02
   9.53810290e-03 -5.41592529e-03]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
ACCURACY => 96.0
PRECISION => 1.0
RECALL => 0.96
AUC => <function auc at 0x7fc16b1d4710>


In [30]:
X_test.shape

(101, 1537)

In [31]:
img_data.columns

Index(['index', 'imdim_0', 'imdim_1', 'imdim_2', 'imdim_3', 'imdim_4',
       'imdim_5', 'imdim_6', 'imdim_7', 'imdim_8',
       ...
       'imdim_759', 'imdim_760', 'imdim_761', 'imdim_762', 'imdim_763',
       'imdim_764', 'imdim_765', 'imdim_766', 'imdim_767', 'img_hash'],
      dtype='object', length=770)

In [32]:
data.shape

(101, 1539)